## Single track model

Searching for a single track model, I found this wikipedia page (which appears to be only in german !?) (https://de.wikipedia.org/wiki/Einspurmodell)

This model assumes a fixed speed v. The vehicle rotates with speed psi_dot. The velocity v is not aligned to the vehicle but has an angle beta towards the main axis.
See below for an image.

First, I've set up a simulation of the dynamics of this system in sympy.
Then, as a second step, I'm deriving a very simple single track model from scratch. Please note: This second model is probably not the canonical "single track model". But I think it would be more comfortable to use.
So my goal is to be able to compare the behaviours.

TODO: so far I'm using a runge-kutta method to step through time. That is not a good idea. Switch to this: https://en.wikipedia.org/wiki/Discretization#Discretization_of_linear_state_space_models


### Part 1, proper single track model

In [1]:
import numpy as np
from sympy import *

In [2]:
from IPython.display import Image
Image(url="https://upload.wikimedia.org/wikipedia/commons/2/24/EinspurKinematik.png")

In [3]:
# parameters of the model
m, theta, l_v, l_h, c_v, c_h, i_S = symbols("m, theta, l_v, l_h, c_v, c_h, i_S")

# some sensible reference values, taken from the wikipedia article
parameters = {
m : 1550,    # kg
theta : 2800, # kg m^2
l_v : 1.344,  # m
l_h : 1.456,  # m
c_v : 75000,  # N / rad
c_h : 150000, # N / rad
i_S : 16
}

In [4]:
# in the single track model, v is assumed to be constant
# here, I'm keeping v symbolic, so that it's possible to plug in different values of v
v = symbols("v")

# state of the model is  x = [beta, psi_dot]
# input of the model is u = [delta / i_S]
beta, psi_dot = symbols("beta psi_dot")
delta = symbols("delta")

In [5]:
# x_dot = A * x + B * u
# here are the corresponding matrices A and B
A = Matrix([[-(c_v + c_h)/(m*v), (m*v**2 - (c_h*l_h - c_v*l_v))/(m*v**2)],
           [- (c_h*l_h - c_v*l_v)/theta, -(c_h*l_h**2 + c_v*l_v**2)/(theta*v)]])

B = Matrix([[-c_v / (m*v)], [c_v*l_v/theta]])

In [6]:
# state space representation: dx = A*x + B*u
def dstate(state, u):
    return A @ state + B@u

In [7]:
# a runge kutta implementation
T = symbols("T")

state = Matrix([beta, psi_dot])
u = Matrix([delta / i_S])

# runge kutta
k1 = dstate(state, u)
k2 = dstate(state + T/2*k1, u)
k3 = dstate(state + T/2*k2, u)
k4 = dstate(state + T*k3, u)

rk = state + T/6 * (k1 + 2*k2 + 2*k3 + k4)

In [8]:
# a sample evaluation of the runge kutta setup

# here we define values to be used for states and inputs
# they will be substituted into the symbolic expression
beta_val = 0
psi_dot_val = 0
delta_val = 0.05
v_val = 10
dT = 1

substitutions = {
    beta: beta_val, 
    psi_dot : psi_dot_val, 
    delta: delta_val, 
    v:v_val,
    T: dT
}

# we also need to substitute the parameters from above
rk.subs({**substitutions, **parameters}).evalf()

Matrix([
[1.89157784341162],
[1.78230636680725]])

### part 2, simplified version

Here, I'm including x and y positions, too.

This allows me to use the dynamics directly, to simulate movement

In [9]:
# time
t = symbols("t")

# parameters are mass and length
# also a parameter is the steering delay
m, l, T_delta = symbols("m l T_delta")
params = [m, l, T_delta]

# to model the state, I'm looking at properties at the back wheel
# speed, x and y positions, as well as the angle beta
# the steering angle delta is also part of the state
v, x, y, beta, delta= symbols("v x y beta delta")
state = [x, y, beta, v, delta]

# input is target steering angle and force for acceleration
delta_target, F = symbols("delta_target F")
u = [delta_target, F]

In [10]:
# angular velocity can be derived from radius of curvature and velocity
# omega * R = v, tan(delta) = l/R
# => 1/R = tan(delta) / l
# => omega = v / R
omega = tan(delta) / l * v

In [11]:
# speed at the center of mass, in the middle of the vehicle, can be derived from speed at back wheel
# and angular velocity
v_mass = sqrt(v**2 + (omega * l/2)**2)

In [12]:
# the velocity of the center of mass can be derived by time
# d_v_mass / d_t = d_v_mass / d_v  * d_v / d_t = a
# -> d_v / d_t = a / d_v_mass / d_v
# d_v / d_t is just what we are looking for
a = F/m
d_v = a / diff(v_mass, v)

In [13]:
# for the dynamics of the steering angle,
# we model a simple PT1 behaviour
d_delta = (delta_target - delta) / T_delta
d_delta

(-delta + delta_target)/T_delta

In [14]:
# the derivatives of beta, x, and y are simple
d_beta = omega
d_x = sin(beta) * v
d_y = cos(beta) * v

In [15]:
# linearization in state space:
d_state = Matrix([d_x, d_y, d_beta, d_v, d_delta])

A = d_state.jacobian(state)
B = d_state.jacobian(u)

In [16]:
import scipy.linalg
# see : https://en.wikipedia.org/wiki/Discretization#Discretization_of_linear_state_space_models
def discretize_AB(mat_a, mat_b, T):
    num_states = mat_a.shape[1]
    num_controls = mat_b.shape[1]
    
    blocked= np.zeros(2*(num_states + num_controls, ))
    blocked[:num_states, :num_states] = np.array(mat_a).astype(np.float64)
    blocked[:num_states, num_states:] = np.array(mat_b).astype(np.float64)
    
    exp_blocked = scipy.linalg.expm(blocked*T)

    mat_a_d = exp_blocked[:num_states, :num_states]
    mat_b_d = exp_blocked[:num_states, num_states:]
    
    return mat_a_d, mat_b_d

In [17]:
param_values = [100, 1, 0.1]
param_subs = list(zip(params, param_values))

current_state = [0, 0, 0, 5, 0]
state_subs = list(zip(state, current_state))

current_inputs = [1, 10]
input_subs = list(zip(u, current_inputs))

all_subs = param_subs + state_subs + input_subs

current_A = A.subs(all_subs).evalf()
current_B = B.subs(all_subs).evalf()

In [18]:
current_A

Matrix([
[0, 0, 5.0,   0,     0],
[0, 0,   0, 1.0,     0],
[0, 0,   0,   0,   5.0],
[0, 0,   0,   0,     0],
[0, 0,   0,   0, -10.0]])

In [19]:
current_B

Matrix([
[   0,    0],
[   0,    0],
[   0,    0],
[   0, 0.01],
[10.0,    0]])

In [20]:
# let's step half a second into the future
T = 0.5
A_d, B_d = discretize_AB(current_A, current_B, T)

next_state = A_d @ current_state + B_d @ current_inputs

In [21]:
next_state

array([2.12331551, 2.5125    , 2.00336897, 5.05      , 0.99326205])